### VARIABLES GLOBALES

In [136]:
import os
from IPython.display import clear_output
from lxml import etree
import webbrowser
#variable global

playlist = etree.parse('playlists.xml')
biblioteca = etree.parse('biblioteca.xml')

### BORRAR LA PANTALLA

In [137]:
def clear():    
    # función que borra la pantalla
    
    # ... si se ejecuta desde un terminal
    os.system('cls' if os.name=='nt' else 'clear')
    # ... si se ejecuta desde jupyter
    clear_output()

### MENU PRINCIPAL

In [138]:
def menu():
    # función que muestra el menu principal
    clear()
    print(""" 
       Bienvenido a nuestro reproductor Olalla's media player.
       aquí podrá, a partir de nuestra biblioteca crear sus propias listas de reproducción
       o utilizar una prehecha,
       ¿Qué desea hacer?
        1. Crear mi propia lista de reproducción
        2. Consultar listas existentes
        3. Escuchar una canción
        4. Salir
        """)

### PAUSAR LA ACCIÓN PARA VER LOS RESULTADOS Y PULSAR ENTER PARA CONTINUAR

In [139]:
def pause():
    #Función que pausa la pantalla para poder ver la información
    programPause = input("Presiona la tecla <ENTER> key para continuar...")

### METODO PARA MOSTRAR TODAS LAS CANCIONES DE LA BIBLIOTECA: PARA CREAR LISTAS DE REPRODUCCIÓN Y ELEGIR CANCIÓN A REPRODUCIR 

In [140]:
def mostrarTodasLasCanciones():
    #Muestro las canciones disponibles con un xsl
    print("Lista total de canciones disponibles en nuestra biblioteca")
    bib_xsl = etree.parse('biblioteca.xsl')
    transform = etree.XSLT(bib_xsl)
    resultadoFinal = transform(biblioteca)
    print(str(resultadoFinal))

### CREAR LISTA DE REPRODUCCIÓN

In [141]:
def crearListaDeReproduccion():
    
    #Mostramos las canciones disponibles
    mostrarTodasLasCanciones()
    
    #Añadimos los datos al archivo de playlist
    nombrePly = input("Introduzca el nombre de su playlist")
    playlistAhora = playlist.xpath('/playlists/playlist')
    
    #Hallamos el id de la playlist lo cual es importante más adelante
    idDePlaylistNueva = len(playlistAhora)+1
    idPly = "pl0"+str(idDePlaylistNueva) 
    
    #Inserto la playlist en el etree en su lugar
    root= playlist.getroot()
    playlistNueva = etree.Element("playlist")
    playlistNueva.attrib["id"]= idPly
    playlistNueva.attrib["nombre"]= nombrePly
    playlistNueva.attrib["modo"]= "aleatorio"
    root.append(playlistNueva)
    
    #Me posiciono en el lugar a insertar las canciones
    meterCancion= root[idDePlaylistNueva-1]
    
    #Meter las canciones en el etree 
    cancionId = input("Introduzca el codigo de las canciones que quiere. pulse return para salir")
    contador=0;
    while cancionId != "":
        meterCancion.append(etree.SubElement(meterCancion, "cancion"))
        meterCancion[contador].attrib["id"]=cancionId
        contador= contador+1
        cancionId = input()
        
    #Grabar el etree en el fichero. Así estará en el etree la próxima vez que carguemos el fichero
    outFile2 = open('playlists.xml','w')
    ficherito = etree.tostring(playlist, pretty_print=True, xml_declaration=True, encoding="utf-8").decode()
    outFile2.write(ficherito)  
    outFile2.close()


### CONSULTAR LISTAS EXISTENTES: MENU OPERAR SOBRE PLAYLIST EXISTENTE

In [142]:
def operarSobrePlaylist():
    print("""
        ¿Qué desea hacer?
            1.Obtener un listado de autores
            2.Obtener un listado de canciones con sus autores y duración
            3.Saber cuántas canciones tiene
            """)

### CONSULTAR LISTAS EXISTENTES: ELECCION DE PLAYLIST Y MENU OPERAR SOBRE PLAYLIST

In [143]:
def consultarListaExistentes():
    
    #Metemos todas las playlist en un diccionario y lo imprimimos
    listas = playlist.xpath('/playlists/playlist')
    dicc_plst = {}
    numero = 0 
    for lista in listas:
        numero=numero+1
        valor = dicc_plst.setdefault(numero,lista.attrib['nombre'])
        print(str(numero)+ "."+lista.attrib['nombre'])
        
    #Elegimos una opcion a hacer    
    opcionMenu2 = int(input("indique de cuál de ellas desea obtener información "))
    eleccionPlst=dicc_plst.get(opcionMenu2)
    
    print(eleccionPlst)
    operarSobrePlaylist()
   
    #Creamos el camino para poder sacar todas las canciones de esa playlist con un xpath y poder usarlo
    #Pedimos que el usuario elija la opcion
    opcion=int(input("elija una opción"))
    cadena1 = '/playlists/playlist[@nombre="'
    cadena2 ='"]/cancion/@id'
    cadenaFinal=cadena1+eleccionPlst+cadena2 
    
    #Las opciones
    if opcion == 1:
        obtenerAutores(cadenaFinal) 
    elif opcion == 2:
        obternerInfoAutores(cadenaFinal, eleccionPlst)
    elif opcion == 3:
        obtenerNumCanciones(cadenaFinal)  
    else:
        print("Opcion incorrecta")
        pause()  
        


### OPCION OBTENER UN LISTADO DE AUTORES DE LA PLAYLIST SELECCIONADA

In [144]:
def obtenerAutores(ply):
    canciones = playlist.xpath(ply)
    listaAutores=[]
    for cancion in canciones:
        camino = '//album[cancion/@id= "'+ cancion + '"]/@artista'
        autores = biblioteca.xpath(camino)
        listaAutores+= autores
    listautoresFinal = set(listaAutores)
    print("Autores:")
   
    for autor in listautoresFinal :
        camino2='//artista[@id="'+ autor + '"]/nombre/text()'
        nombres = biblioteca.xpath(camino2)[0]
        print("*"+nombres)
     
    pause()

### OBTENER UN LISTADO DE CANCIONES CON SUS AUTORES Y DURACIÓN

In [145]:
def obternerInfoAutores(ply, nombre):
    canciones2= playlist.xpath(ply)
    totalCanciones=len(canciones2)
   
    #creo un nuevo fichero sólo con las canciones que quiero de la playlist seleccionada y lo relleno
    page = etree.Element("playlists")
    doc = etree.ElementTree(page)
    page.append(etree.Element("playlist"))
    page[0].attrib["nombre"]=nombre
    for i in range(0, totalCanciones):
        page[0].append(etree.SubElement(page[0], "cancion"))
    contador=0
    for cancion2 in canciones2:
        page[0][contador].text = cancion2
        contador= contador+1
    #Lo grabo
    outFile = open('playlist2.xml','w')
    ficherito = etree.tostring(doc, pretty_print=True, xml_declaration=True, encoding="utf-8").decode()
    outFile.write(ficherito)  
    outFile.close()
    
    #Le aplico un xsl con el que saco la info
    ply2 = etree.parse('playlist2.xml')
    ply2_xsl = etree.parse('playlist2.xsl')
    transform = etree.XSLT(ply2_xsl)
    resultado = transform(ply2)
    print(str(resultado))
    pause()

### OBTENER NUMERO DE CANCIONES

In [146]:
def obtenerNumCanciones(ply):   
    canciones3 = playlist.xpath(ply)
    NumCanciones = len(canciones3)
    print("El numero de canciones en la lista "+ str(NumCanciones))
    pause()

### ESCUCHAR CANCION

In [147]:
def escucharCancion():
    #Mostramos las canciones disponibles
    mostrarTodasLasCanciones()
   
    #le pedimos que nos meta el codigo de la cancion
    cancRepr = input("Introduzca el codigo de la cancion a reproducir")
    
    #Creamos media cadena para el xpath y metemos en xpath
    mitadCadena ='//cancion [@id="'+ cancRepr
    otraMitad= '//album[cancion/@id="'+ cancRepr
    #creamos un nuevo xml que traduciremos con un xsl para crear el reproductor
    canc = etree.Element("cancion")
    doc1 = etree.ElementTree(canc)
    canc.append(etree.SubElement(canc, "nombre"))
   
    nombre = biblioteca.xpath(mitadCadena +'"]/titulo/text()')
    canc[0].text=nombre[0]
    canc.append(etree.SubElement(canc, "album"))
    Album = biblioteca.xpath(otraMitad +'"]/titulo/text()')
    canc[1].text = Album[0]
    canc.append(etree.SubElement(canc, "duracion"))
    Duracion = biblioteca.xpath(mitadCadena + '"]/@duracion')
    canc[2].text = Duracion[0]
   
    #Lo grabamos en el fichero reproductor
    outFile3 = open('reproductor.xml','w')
    ficherito3 = etree.tostring(doc1, pretty_print=True, xml_declaration=True, encoding="utf-8").decode()
    outFile3.write(ficherito3)  
    outFile3.close()
    
    #Lo transformamos con el xsl y mostramos el resultado en el navegador
    cancioncilla= etree.parse('reproductor.xml')
    cancioncilla_xsl = etree.parse('reproductor.xsl')
    transform4 = etree.XSLT(cancioncilla_xsl)
    resultado4 = transform4(cancioncilla) 
    f = open('reproductor.html','w')
    f.write(str(resultado4))
    f.close()
    webbrowser.open_new_tab('reproductor.html')

### MENÚ PRINCIPAL DE TODO

In [148]:
while True:

    # Mostramos el menu
    menu()
    
    # Introducir opción
    opcionMenu = int(input("Seleccionar opcion [1-4]"))

    if opcionMenu == 1:
        crearListaDeReproduccion()
    elif opcionMenu == 2:
        consultarListaExistentes()
    elif opcionMenu == 3:
        escucharCancion()
    elif opcionMenu == 4:
        print('Adios')
        break
    else:
        print("Opcion incorrecta")
        pause()

 
       Bienvenido a nuestro reproductor Olalla's media player.
       aquí podrá, a partir de nuestra biblioteca crear sus propias listas de reproducción
       o utilizar una prehecha,
       ¿Qué desea hacer?
        1. Crear mi propia lista de reproducción
        2. Consultar listas existentes
        3. Escuchar una canción
        4. Salir
        
Seleccionar opcion [1-4]4
Adios
